In [4]:
 def load(filePath):
    with open(filePath, 'rb') as f:
        data = f.read()
        k = safe_load(data)
        
    return k

def dbConnection(host, user, password, database=""):
    import mysql.connector
    host=host
    username=user
    password=password
    database=database
    mydb = mysql.connector.connect(host="{0}".format(host), username="{0}".format(username),password="{0}".format(password),database="{0}".format(database))
    return mydb

def removeDL(dict, remove):
    lst=list(dict.keys())
    lst.remove(remove)
    return lst

def connection(hosts, group):
    h=[]
    for i in hosts:
        if i['group']==group:
            host=removeDL(dict=i,remove="group")
            
            for j in host:
                if "database" in list(i[j].keys()):
                    h.append(dbConnection(i[j]["host"],i[j]["user"],i[j]["passwd"],i[j]["database"]))
                else:
                    h.append(dbConnection(i[j]["host"],i[j]["user"],i[j]["passwd"]))
            return h
    
def host(group):
    hosts= load("hosts.yml")
    hostsCon=connection(hosts, group)
    return hostsCon

def checkKey(dict, key):      
    if key in dict.keys(): 
        return True 
    else: 
        return False 
    
def listtostring(sep, convert):
    sep=sep.join(convert)
    return sep

def create(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        if tasks["type"]=="database":
            mycursor = i.cursor()
            mycursor.execute("SHOW DATABASES")
            flag=1
            for x in mycursor:
                if x[0]== tasks["name"]:
                    flag=0
                    print("Database {} already exists".format(tasks["name"]))
            if flag == 1:                         
                exe=i.cursor()
                exe.execute("CREATE DATABASE {0}".format(tasks["name"]))
                print("Database {} created Successfully".format(tasks["name"]))
        if tasks["type"]=="table":
            mycursor = i.cursor()
            mycursor.execute("SHOW TABLES")
            flag=1
            for x in mycursor:
                if x[0]==tasks["name"]:
                    flag=0
                    print("Table {} already exists".format(tasks["name"]))
            if flag==1:
                if checkKey(tasks, "columns"):
                    sep=","
                    sep=listtostring(sep,tasks["columns"])                                    
                    exe=i.cursor()
                    exe.execute("CREATE TABLE {0}( {1} )".format(tasks["name"], sep) )
                    print("Table {} Created Successfully".format(tasks["name"]))
                else:
                    print("ERROR: A table must have at least 1 column, {} has no column".format(tasks["name"]))
        
        if tasks["type"]=="role":
            mycursor = i.cursor()
            #print(tasks)
            for r in tasks["role"]:
                print(r)
                s= "create role if not exists {}".format(r)
                mycursor.execute(s)
                print(" Role Created Successfull")
                for x in mycursor:
                    print(x)
                    
def insert(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        print("tasks:", tasks)
        print("group:", group)
        if checkKey(tasks,"columns"):
            print("yes")
            columns=listtostring(",", tasks["columns"])
            #sptask=tasks["values"].split("--")
            sptask=tasks["values"]
            #if sptask[0]=='':
            mycursor = i.cursor()
            for h in sptask[1:]:
                s="insert into {0} ({1}) values({2})".format(tasks["table"],columns, h)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record inserted.")
           
            
        else:
            print("no")            
            #sptask=tasks["values"].split("--")
            sptask=tasks["values"]
            #if sptask[0]=='':
            mycursor = i.cursor()
            for h in sptask:
                s="insert into {} values({})".format(tasks["table"],h)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record inserted.")
           
        
        
def alter(tasks, group):
    print("tasks:", tasks)
    print("group:", group)
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        if checkKey(tasks,"add"):
            for l in tasks["add"]:
                s="alter table {0}  add {1}".format(tasks["table"], l)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record altered.")
        if checkKey(tasks,"drop"):
            for l in tasks["drop"]:
                s="alter table {0}  drop column {1}".format(tasks["table"], l)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record altered.")
        if checkKey(tasks,"modify"):
            for l in tasks["modify"]:
                s="alter table {0}  modify {1}".format(tasks["table"], l)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record altered.")
        if checkKey(tasks,"renameColumn"):
            for l in tasks["renameColumn"]:
                s="alter table {0} change column {1}".format(tasks["table"], l)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record altered.")
        if checkKey(tasks,"renameTable"):
            for l in tasks["renameTable"]:
                s="alter table {0}  rename {1}".format(tasks["table"], l)
                mycursor.execute(s)
                i.commit()
                print(mycursor.rowcount, "record altered.")
                    
            
            

def command(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        print("tasks", tasks)
        mycursor = i.cursor()
        for q in tasks["query"]:
            mycursor.execute(q)
            if tasks["commit"]:
                i.commit()
            for x in mycursor:
                print(x)
                
                
def grant(tasks, group):
    #print("tasks:", tasks)
    #print("group:", group)
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        if tasks["type"]=="role":
            s='grant "' + listtostring('","',tasks["roles"]) +'" to "' + listtostring('","' ,tasks["user_or_role"]) +'"'
            mycursor.execute(s)            
            print("Grant roles successful")
        
        if tasks["type"]=="privileges":
            s='grant ' + listtostring(',', tasks["privileges"]) + ' on {0}.{1} to  "'.format(tasks["database"], tasks["table"]) + listtostring('","' ,tasks["user_or_role"]) +'"'
            mycursor.execute(s)            
            print("Grant permission successful")
        
        if tasks["type"]=="proxy":
            if tasks["grant_option"]:                
                s='grant proxy on "' + listtostring('","',tasks["proxy_user_or_role"]) +'" to "' + listtostring('","' ,tasks["user_or_role"]) +'" with grant option'
                print(s)
                mycursor.execute(s)            
                print("Grant proxy successful")
            else:
                s='grant proxy on "' + listtostring('","',tasks["proxy_user_or_role"]) +'" to "' + listtostring('","' ,tasks["user_or_role"]) +'"'
                print(s)
                mycursor.execute(s)            
                print("Grant proxy successful with grant option")
            
            
        if checkKey(tasks,"roles") & checkKey(tasks,"permission"):
            print("Error: you can have either permission or roles set, not both")
            
            
def drop(tasks, group):
    print("tasks:", tasks)
    print("group:", group)
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()        
        if tasks["type"]=="table":            
            s='drop table IF EXISTS {0}'.format(listtostring(',',tasks["name"]))
            if checkKey(tasks, "temporary"):                
                if tasks["temporary"]:
                    s='drop temporary table IF EXISTS {0}'.format(listtostring(',',tasks["name"]))            
            if checkKey(tasks, "restrict"):
                if tasks["restrict"]:
                    s=s+' restrict'
            if checkKey(tasks, "cascade"):
                if tasks["cascade"]:
                    s=s+' cascade'
            print(s)
            mycursor.execute(s)            
            print("drop table successful")
            
        if tasks["type"]=="index":            
            s='drop index  {0} on {1}'.format(listtostring(',',tasks["index_name"]), listtostring(',',tasks["table_name"]))            
            if checkKey(tasks, "algorithm"):
                s=s+' algorithm = {}'.format(tasks["algorithm"])
            if checkKey(tasks,"lock"):
                s=s+' lock = {}'.format(tasks["lock"])
            print(s)
            mycursor.execute(s)            
            print("drop index successful")
            
        if tasks["type"]=="logfile":            
            s='drop logfile group  {0}'.format(listtostring(',',tasks["logfile_group"]))            
            if checkKey(tasks, "engine"):
                s=s+' engine = {}'.format(tasks["engine"])
            print(s)
            mycursor.execute(s)            
            print("drop logfile successful")
            
        if tasks["type"]=="database":            
            s='drop database IF EXISTS {0}  '.format(listtostring(',',tasks["name"]))              
            print(s)    
            mycursor.execute(s)            
            print("drop database successful")
        if tasks["type"]=="schema":            
            s='drop schema IF EXISTS {0}  '.format(listtostring(',',tasks["name"]))              
            print(s)  
            mycursor.execute(s)            
            print("drop schema successful")
        if tasks["type"]=="function":            
            s='drop function IF EXISTS {0}  '.format(listtostring(',',tasks["name"]))              
            print(s)
            mycursor.execute(s)            
            print("drop function successful")
        if tasks["type"]=="procedure":            
            s='drop procedure IF EXISTS {0}  '.format(listtostring(',',tasks["name"]))              
            print(s) 
            mycursor.execute(s)            
            print("drop procedure successful")
        if tasks["type"]=="server":            
            s='drop server IF EXISTS {0}  '.format(listtostring(',',tasks["server_name"]))              
            print(s) 
            mycursor.execute(s)            
            print("drop server successful")
        if tasks["type"]=="spatial_reference_system": 
            for t in tasks["srid"]:
                s='drop spatial reference system IF EXISTS {0} '.format(t)              
                print(s)
                mycursor.execute(s)            
            print("drop spacial reference system successful")
            
        if tasks["type"]=="tablespace":            
            s='drop tablespace  {0}'.format(listtostring(',',tasks["tablespace_name"]))
            if checkKey(tasks, "undo"):
                if tasks["undo"]:
                    s='drop undo tablespace  {0}'.format(listtostring(',',tasks["tablespace_name"]))
            if checkKey(tasks, "engine"):
                s=s+' engine = {}'.format(tasks["engine"])           
            print(s)
            mycursor.execute(s)            
            print("drop tablespace successful")
            
        if tasks["type"]=="trigger":            
            s='drop trigger IF EXISTS {0} '.format(listtostring(',',tasks["trigger_name"]))           
            if checkKey(tasks, "schema_name"):
                for t in tasks["trigger_name"]:
                    s='drop trigger IF EXISTS {0}.{1} '.format(tasks["schema_name"], t )
                    print(s)   
                    mycursor.execute(s)            
                    print("drop trigger successful")
        
        if tasks["type"]=="view":            
            s='drop view IF EXISTS {0}'.format(listtostring(',',tasks["view_name"]))
            if checkKey(tasks, "restrict"):
                if tasks["restrict"]:
                    s=s+' restrict'
            if checkKey(tasks, "cascade"):
                if tasks["cascade"]:
                    s=s+' cascade'
            print(s)
            mycursor.execute(s)            
            print("drop view successful")
            
        if tasks["type"]=="event":            
            s='drop event IF EXISTS {0}  '.format(listtostring(',',tasks["event_name"]))              
            print(s)
            mycursor.execute(s)            
            print("drop event successful")
        if tasks["type"]=="user":            
            s='drop user IF EXISTS {0}  '.format(listtostring(',',tasks["user_name"]))              
            print(s)
            mycursor.execute(s)            
            print("drop user successful")
        if tasks["type"]=="role":            
            s='drop role IF EXISTS {0}  '.format(listtostring(',',tasks["name"]))              
            print(s)
            mycursor.execute(s)            
            print("drop role successful")

            
def revoke(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        if tasks["type"]=="role":
            s='revoke "' + listtostring('","',tasks["roles"]) +'" from "' + listtostring('","' ,tasks["user_or_role"]) +'"'
            mycursor.execute(s)            
            print("revoke roles successful")
        
        if tasks["type"]=="privileges":
            s='revoke ' + listtostring(',', tasks["privileges"]) + ' on {0}.{1} from  "'.format(tasks["database"], tasks["table"]) + listtostring('","' ,tasks["user_or_role"]) +'"'
            mycursor.execute(s)            
            print("revoke permission successful")
        
        if tasks["type"]=="proxy":            
            s='revoke proxy on "' + listtostring('","',tasks["proxy_user_or_role"]) +'" from "' + listtostring('","' ,tasks["user_or_role"]) +'"'
            print(s)
            mycursor.execute(s)            
            print("revoke proxy successful with grant option")
            
            
        if checkKey(tasks,"roles") & checkKey(tasks,"permission"):
            print("Error: you can have either permission or roles set, not both")
            
def truncate(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        for t in tasks["table_name"]:           
            if checkKey(tasks, "database"):
                s="truncate table {0}.{1}".format(tasks["database"],t)
            else:
                 s="truncate table {}".format(t)
            print(s)
            mycursor.execute(s)            
            print("truncate successfull")
            
def lock(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        print(tasks)
        if checkKey(tasks, "database"):            
            for t in range(len(tasks["table_name"])):
                s= "lock table {0}.{1} {2}".format(tasks["database"], tasks["table_name"][t], tasks["lock_type"][t])
                mycursor.execute(s)
                print("lock successfull")
        else:
            for t in range(len(tasks["table_name"])):
                s= "lock table {0} {1}".format(tasks["table_name"][t], tasks["lock_type"][t])
                mycursor.execute(s)
                print("lock successfull")
        
        
def rename(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        print(tasks)
        for t in range(len(tasks["old_table_name"])):
            s="rename table {} to {}".format(tasks["old_table_name"][t], tasks["new_table_name"][t])
            print(s)
            mycursor.execute(s)            
            print("rename successfull")
            
def update(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        s="update"
        if checkKey(tasks, "low_priority"):
            s=s+" low_priority "
        if checkKey(tasks, "ignore"):
            s=s+" ignore "
        s=s+" {0} set {1} ".format(listtostring(", ",tasks["table_reference"]), listtostring(", " , tasks["assignment_list"]))
        #print(s)
        if checkKey(tasks, "where_condition"):
            s=s+ " where {0} ".format(listtostring(", ", tasks["where_condition"]))
        if len(tasks["table_reference"]) ==1:
            if checkKey(tasks,"order_by"):
                s=s+ " order by {} ".format(tasks["order_by"])
            if checkKey(tasks, "limit_row_count"):
                s=s+ " limit {} ".format(tasks["limit_row_count"])
        print(s)
        mycursor.execute(s)            
        print("update successfull")
        
        
        
        
def delete(tasks, group):
    dbhost= host(group)
    for i in dbhost:
        mycursor = i.cursor()
        #print(tasks)
        s="delete"
        if checkKey(tasks, "low_priority"):
            s=s+" low_priority "
        if checkKey(tasks, "ignore"):
            s=s+" ignore "
        if checkKey(tasks, "quick"):
            s=s+" quick "
        if checkKey(tasks, "from_table_reference"):
            s=s+" {} from {} ".format(listtostring(", ", tasks["table_reference"]), listtostring(", ", tasks["from_table_reference"]))
            print(s)
            
        elif checkKey(tasks, "using_table_reference"):
            s=s+" from {} using {} ".format(listtostring(", ", tasks["table_reference"]), listtostring(", ", tasks["using_table_reference"]))
            
            
        else:
            s=s+" from {0} ".format(tasks["table_reference"][0])
            print(s)
            if checkKey(tasks, "partition_name_list"):
                s=s+ " partition {} ".format(listtostring(", ", tasks["partition_name_list"]))
        if checkKey(tasks, "where_condition"):
            s=s+ " where {0} ".format(listtostring(", ", tasks["where_condition"]))
        if len(tasks["table_reference"])==1:
            if checkKey(tasks,"order_by"):
                s=s+ " order by {} ".format(tasks["order_by"])
            if checkKey(tasks, "limit_row_count"):
                s=s+ " limit {} ".format(tasks["limit_row_count"])
        print(s)
        mycursor.execute(s)            
        print("update successfull")
        """
        DELETE [LOW_PRIORITY] [QUICK] [IGNORE]
    tbl_name[.*] [, tbl_name[.*]] ...
    FROM table_references
    [WHERE where_condition]

DELETE [LOW_PRIORITY] [QUICK] [IGNORE]
    FROM tbl_name[.*] [, tbl_name[.*]] ...
    USING table_references
    [WHERE where_condition]
        DELETE [LOW_PRIORITY] [QUICK] [IGNORE] FROM tbl_name [[AS] tbl_alias]
    [PARTITION (partition_name [, partition_name] ...)]
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]
    """


from yaml import safe_load
with open("create.yml", 'rb') as f:
    data = f.read()
    k = safe_load(data)
    


for i in range(len(k)):
    for j in range(len(k[i]["tasks"])):
        l=[]
        l=list(k[i]["tasks"][j].keys())
        l.remove('name')
        #__import__("{0}1.{1}.run".format(k[0]["dbName"],l[0]), fromlist=["run"])
        #print("EXECUTING TASKS: ", k[i]["tasks"]["name"])
        if l[0]=="create":
            create(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="insert":
            insert(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="alter":
            alter(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="grant":
            grant(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="select":
            select(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="drop":
            drop(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="truncate":
            truncate(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="revoke":
            revoke(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="lock":
            lock(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="command":
            command(k[i]["tasks"][j][l[0]],k[i]["group"])            
        if l[0]=="rename":
            rename(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="update":
            update(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="delete":
            delete(k[i]["tasks"][j][l[0]],k[i]["group"])
        if l[0]=="select":
            select(k[i]["tasks"][j][l[0]],k[i]["group"])
        
            


Database test8 created Successfully
Table ttt8 Created Successfully
admin
 Role Created Successfull
student
 Role Created Successfull
'automation'@'localhost'
 Role Created Successfull
Database test22 created Successfully
Table ttt212 Created Successfully
admin
 Role Created Successfull
student
 Role Created Successfull
'automation'@'localhost'
 Role Created Successfull
